In [2]:
import emcee
import arviz as az
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
import time
import jax.numpy as jnp
#from functions.distributions import log_gaussian_2d
import pandas as pd

In [3]:
class MetropolisHastings:
    def __init__(self, target_distr, initial_state=None, seed=None):
        """
        Initialize the Metropolis-Hastings sampler.
        
        Parameters:
        - target_distr: function
            The target distribution from which we want to sample.
        - initial_state: array-like, optional
            The initial state of the Markov chain. Default is a zero vector.
        - seed: int, optional
            Random seed for reproducibility. Default is None.
        """
        self.target_distr = target_distr
        self.dim = len(initial_state) if initial_state is not None else 1
        self.initial_state = np.array(initial_state) if initial_state is not None else np.zeros(self.dim)
        self.seed = seed
        self.samples = []
        self.acceptance_rate = 0.0
        self.mean = None
        self.covariance = None
        self.time = 0.0
        
        if seed is not None:
            np.random.seed(seed)
    
    def sample(self, n_samples):
        """
        Generate samples using the Metropolis-Hastings algorithm.
        
        Parameters:
        - n_samples: int
            The number of samples to generate.
        
        Returns:
        - samples: np.array
            Array of generated samples.
        """
        start_time = time.time()
        
        proposal_distr = scipy.stats.multivariate_normal
        # Random covariance matrix for the proposal distribution
        A = np.random.rand(self.dim, self.dim)
        cov = np.dot(A, A.T)
        
        x_t = self.initial_state
        samples = []
        accept_count = 0
        
        while len(samples) < n_samples:
            x_prime = proposal_distr.rvs(mean=x_t, cov=cov)
            acceptance_ratio = (self.target_distr(x_prime) * proposal_distr.pdf(x_t, mean=x_prime, cov=cov)) / \
                               (self.target_distr(x_t) * proposal_distr.pdf(x_prime, mean=x_t, cov=cov))
            
            acceptance_ratio = min(1, acceptance_ratio)
            if np.random.uniform(0, 1) < acceptance_ratio:
                x_t = x_prime
                samples.append(x_t)
                accept_count += 1
            else:
                samples.append(x_t)
        
        self.samples = np.array(samples)
        self.acceptance_rate = accept_count / n_samples
        self.mean = np.mean(self.samples, axis=0)
        self.covariance = np.cov(self.samples.T)
        
        self.time = time.time() - start_time
        
        return self.samples

2D Gaussian Analysis of Metropolis Hasting

In [4]:
# Define the target distribution: a 2D Gaussian
def target_distribution(x):
    mean = np.array([1, -1.5])
    cov = np.array([[4, 0.7], [0.7, 1.5]])
    return scipy.stats.multivariate_normal.pdf(x, mean=mean, cov=cov)

# Initialize the Metropolis-Hastings sampler
initial_state = np.zeros(2)
MH_sampler_2D = MetropolisHastings(target_distribution, initial_state, seed=42)

# Sample sizes to test
sample_sizes = [10000, 50000, 100000]
results = {}

# Sampling procedure
for size in sample_sizes:
    MH_sampler_2D.sample(size)
    samples_with_acceptance = np.insert(MH_sampler_2D.samples, 0, MH_sampler_2D.acceptance_rate, axis=0)
    results[size] = samples_with_acceptance

# Save the results

for size in sample_sizes:
    df = pd.DataFrame(results[size])
    filename = f"MH_samples_2D_{size}.csv"
    df.to_csv(filename, index=False)

15D Gaussian Analysis of Metropolis Hasting

In [5]:
import numpy as np
import matplotlib.pyplot as plt

# Define the target distribution: a multivariate Gaussian shifted from zero with some correlation
mean = np.ones(15)  # Mean is shifted from zero
cov = np.eye(15) * 0.5 + 0.5  # Correlated covariance matrix

def target_distribution(x):
    return scipy.stats.multivariate_normal.pdf(x, mean=mean, cov=cov)

# Initialize the Metropolis-Hastings sampler
initial_state = np.zeros(15)
MH_sampler_15D = MetropolisHastings(target_distr=target_distribution, initial_state=initial_state, seed=42)

# Sample sizes to test
sample_sizes = [10000, 50000, 100000]
results = {}

# Sampling procedure
for size in sample_sizes:
    MH_sampler_15D.sample(size)
    samples_with_acceptance = np.insert(MH_sampler_15D.samples, 0, MH_sampler_15D.acceptance_rate, axis=0)
    results[size] = samples_with_acceptance

# Save the results

for size in sample_sizes:
    df = pd.DataFrame(results[size])
    filename = f"MH_samples_15D_{size}.csv"
    df.to_csv(filename, index=False)

Rosenbrock 2D Analysis of Metropolis Hasting

In [6]:
import numpy as np
import matplotlib.pyplot as plt

# Define the Rosenbrock function
def rosenbrock(x):
    return sum(100.0 * (x[1:] - x[:-1]**2.0)**2.0 + (1 - x[:-1])**2.0)

# Transform the Rosenbrock function into a probability density function
def rosenbrock_pdf(x):
    return np.exp(-rosenbrock(x))

# Initialize the Metropolis-Hastings sampler
initial_state = np.zeros(2)  # Start from the origin for simplicity
MH_sampler_RB = MetropolisHastings(target_distr=rosenbrock_pdf, initial_state=initial_state, seed=42)

# Sample sizes to test
sample_sizes = [10000, 50000, 100000]
results = {}

# Sampling procedure
for size in sample_sizes:
    MH_sampler_RB.sample(size)
    samples_with_acceptance = np.insert(MH_sampler_RB.samples, 0, MH_sampler_RB.acceptance_rate, axis=0)
    results[size] = samples_with_acceptance

# Save the results

for size in sample_sizes:
    df = pd.DataFrame(results[size])
    filename = f"MH_samples_RB_{size}.csv"
    df.to_csv(filename, index=False)
